# 特征工程——4——20 1208 16

1. 将训练集与测试集的统计特征分离 跑一遍试试效果
2. 增加统计特征(max min median std) 在特征工程_2中做
3. 找train和test中差异比较大的fea， 在特征工程_3中做  （删除了revolUtil与issueDate看看效果）
4. 将nan太多的行删了，在特征工程_4做

In [109]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import datetime
import warnings
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import roc_auc_score
warnings.filterwarnings('ignore')
pd.set_option('max_columns', None)

In [110]:
# reduce_mem_usage 函数通过调整数据类型，帮助我们减少数据在内存中占用的空间
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() 
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
#         else:
#             df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() 
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [126]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('testA.csv')

In [127]:
#合并训练集测试集便于操作
data_train['istest'] = 0
data_test['istest'] = 1
# data = pd.concat([data_train, data_test])
# del data_train
# del data_test
# data = reduce_mem_usage(data)

In [129]:
data_train[data_train.isnull().sum(axis=1) > 6]


,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,istest
1,1,18000.0,5,18.49,461.90,D,D2,219843.0,5 years,0,46000.0,2,2012-08-01,0,0,156.0,18,27.83,0.0,700.0,704.0,13.0,0.0,0.0,15096.0,38.9,18.0,1,0,May-2002,1723.0,1.0,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,0
30,30,10000.0,3,14.59,344.65,D,D1,241561.0,4 years,1,40000.0,0,2010-04-01,0,0,56.0,8,24.15,0.0,675.0,679.0,8.0,0.0,0.0,22778.0,87.6,30.0,1,0,Dec-1995,15383.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
63,63,3000.0,3,7.90,93.88,A,A4,222245.0,< 1 year,0,96000.0,1,2011-11-01,0,8,0.0,0,0.19,1.0,735.0,739.0,4.0,0.0,0.0,25.0,0.1,14.0,1,0,Jul-1996,3581.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
91,91,15000.0,5,18.25,382.95,D,D5,220161.0,10+ years,0,50004.0,1,2011-12-01,1,0,495.0,14,21.41,1.0,685.0,689.0,10.0,0.0,0.0,14020.0,47.7,29.0,1,0,Oct-1989,269.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
92,92,20000.0,3,13.99,683.46,C,C1,182720.0,10+ years,0,82000.0,0,2012-03-01,1,0,408.0,14,15.36,0.0,700.0,704.0,8.0,0.0,0.0,15988.0,71.1,32.0,1,0,Mar-1999,38.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799890,799890,6000.0,3,12.73,201.40,C,C1,242702.0,NaN,0,60000.0,0,2010-01-01,0,10,420.0,4,12.80,0.0,680.0,684.0,7.0,1.0,0.0,7088.0,63.3,12.0,1,0,Oct-2001,16525.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
799934,799934,9000.0,5,12.99,204.74,C,C1,229597.0,5 years,1,60000.0,0,2011-06-01,0,3,156.0,18,15.90,0.0,725.0,729.0,10.0,0.0,0.0,4290.0,19.4,25.0,1,0,Feb-1994,7113.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
799962,799962,35000.0,3,7.90,1095.16,A,A4,343348.0,10+ years,2,155000.0,2,2012-06-01,0,4,56.0,8,7.43,0.0,750.0,754.0,6.0,0.0,0.0,28624.0,66.7,25.0,1,0,Aug-1992,95.0,1.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,0
799966,799966,15000.0,3,7.91,469.43,A,A4,218781.0,10+ years,0,160000.0,2,2011-06-01,0,0,186.0,9,9.38,0.0,730.0,734.0,16.0,0.0,0.0,17070.0,24.4,25.0,1,0,Mar-1993,193.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [79]:
#处理employmentLength
employment_map = {}
for i in range(2, 10):
    employment_map[f'{i} years'] = i
employment_map['10+ years'] = 10
employment_map['< 1 year'] = 0
employment_map['1 year'] = 1
print(employment_map)
data['employmentLength'] = data['employmentLength'].map(employment_map)
data['employmentLength'].fillna(0, inplace=True)

{'2 years': 2, '3 years': 3, '4 years': 4, '5 years': 5, '6 years': 6, '7 years': 7, '8 years': 8, '9 years': 9, '10+ years': 10, '< 1 year': 0, '1 year': 1}


In [80]:
from datetime import datetime

def month_dif(a, b):
    return abs((a.year-b.year)*12 + (a.month-b.month))


def f(x):
    t = datetime.strptime(x, '%b-%Y')
    n = datetime.today()
    return month_dif(t, n)


data['earliesCreditLine'] = data['earliesCreditLine'].apply(f)
data['earliesCreditLine'].value_counts()

232    6957
220    6779
207    6710
230    6636
195    6556
       ... 
753       1
752       1
62        1
730       1
899       1
Name: earliesCreditLine, Length: 726, dtype: int64

In [81]:
def f(x):
    if x == 54:
        return 1
    else:
        return 0

data['employmentTitle_bin'] = data['employmentTitle'].apply(f)
data['employmentTitle_bin'].value_counts()

0    936022
1     63978
Name: employmentTitle_bin, dtype: int64

In [82]:
tmp = data['subGrade'].value_counts()
value = sorted(tmp.keys())
subgrade_num = len(value)
key = list(range(1, subgrade_num+1))
d = dict(zip(value, key))
data['subGrade']= data['subGrade'].map(d)

In [83]:
# Acc处理
data['Acc'] = data['totalAcc'] - data['openAcc']

In [84]:
#处理postCode和regionCode
# t = data['regionCode'].value_counts().reset_index()
# t.columns = ['regionCode', 'count']
# t['cumsum'] = t['count'].cumsum()
# t
data['postCode_bin'] = pd.qcut(data['postCode'], 10, labels=list(range(10)))
# data['regionCode'] = pd.qcut(data['regionCode'], 12, labels=list(range(12)))

In [86]:
from scipy.stats import ks_2samp
for fea in data.columns:
    a = data[data['istest']==0][fea]
    b = data[data['istest']==1][fea]
    stat, pvalue = ks_2samp(a, b)
    if pvalue<0.1:
        print(f'alert: {fea} is abnormal')
# 检测出来 issueDate 与 revolUtil列，在训练集和测试集中分布不一致

alert: id is abnormal
alert: issueDate is abnormal
alert: isDefault is abnormal
alert: revolUtil is abnormal
alert: istest is abnormal


## 制作统计特征

In [106]:
def add_statistical_fea(data, groupby, features, methods):
    l = []
    for i in features:
        for j in methods:
            l.append([i, j])
    for fea, method in tqdm(l):
        new_fea = fea + '_' + method
        train = data[data['istest']==0]
        train[new_fea] = train.groupby(groupby)[fea].transform(method)
        train_to_dct = train[[groupby, new_fea]].drop_duplicates()
        dct = dict(zip(train_to_dct[groupby], train_to_dct[new_fea]))
        test = data[data['istest']==1]
        test[new_fea] = test[groupby].map(dct)
        data = pd.concat([train, test])
    return data

In [107]:
num_fea_to_statistical = ['installment', 'dti', 'revolBal',
                          'earliesCreditLine', 'ficoRangeLow', 'n6', 'totalAcc', 
                          'n9', 'n8', 'n5']
cat_fea_to_statistical = ['postCode', 'employmentTitle', 'title']

In [108]:
data = add_statistical_fea(data, 'subGrade', num_fea_to_statistical, ['median', 'min', 'max', 'std', 'mean'])
data = add_statistical_fea(data, 'subGrade', cat_fea_to_statistical, ['count'])

  4%|▍         | 2/50 [00:10<04:16,  5.34s/it]


KeyboardInterrupt: 

In [90]:
fea_to_del = ['grade', 'ficoRangeHigh', 'homeOwnership', 'regionCode', 'policyCode',
             'n1', 'n2', 'n3', 'n7', 'n10', 'loanAmnt', 'interestRate', 
             'pubRec', 'pubRecBankruptcies',
             'revolUtil', 'issueDate']
data.drop(columns=fea_to_del, inplace=True)

In [91]:
save_path = './results/'
data.to_csv(save_path + 'data_3.csv')

In [71]:
len(data.columns)

94

In [45]:
important_fea = ['id', 'istest', 'isDefault',
    'term',
 'n4_mean',
 'employmentLength',
 'openAcc',
 'n4',
 'n5',
 'totalAcc',
 'n14',
 'title',
 'subGrade',
 'Acc',
 'n8',
 'n9',
 'ficoRangeLow',
 'n6',
 'purpose',
 'revolUtil',
 'dti',
 'postCode',
 'installment',
 'annualIncome',
 'employmentTitle',
 'revolBal',
 'earliesCreditLine']

In [46]:
data[important_fea].to_csv(save_path + 'data_important.csv')

In [24]:
# data = data[fea_sel]
data = reduce_mem_usage(data)
train = data[data['istest']==0].drop(columns=['istest'])
y_train = train['isDefault']
X_train = train.drop(columns=['isDefault'])
X_test = data[data.istest==1].drop(columns=['istest', 'isDefault'])
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

Memory usage of dataframe is 113000000.00 MB
Memory usage after optimization is: 113000000.00 MB
Decreased by 0.0%
(800000, 50)
(200000, 50)
(800000,)


# 以上是特征工程 

In [25]:
X = X_train
y = y_train

In [27]:
from sklearn.model_selection import StratifiedKFold
# 分离数据集，方便进行交叉验证

# 10折交叉验证
folds = 5
seed = 2020
skf = StratifiedKFold(n_splits=folds)

In [28]:
X_train.drop(columns=['id'])

,term,installment,subGrade,employmentTitle,employmentLength,annualIncome,verificationStatus,purpose,postCode,dti,delinquency_2years,ficoRangeLow,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,n0,n4,n5,n6,n8,n9,n11,n12,n13,n14,employmentTitle_bin,fico,Acc,remainRec,remainRec_bin,pubRec_bin,homeOwnership_0,homeOwnership_1,debtRatio,postCode_bin,n6_mean,dti_mean,revolUtil_mean,n9_mean,n4_mean,n11_mean,ficoRangeLow_mean
0,5,918.0000,22,320.0,2.0,110000.0,2,1,137.0,17.046875,0.0,730.0,7.0,0.0,0.0,24178.0,48.90625,27.0,0,0,232,1.0,0.0,4.0,9.0,8.0,12.0,2.0,0.0,0.0,0.0,2.0,0,1,20.0,0.0,0,0,0,0,0.100142,3.0,8.851562,20.609375,59.15625,6.007812,4.605469,0.001111,682.0
1,5,462.0000,17,219843.0,5.0,46000.0,2,0,156.0,27.828125,0.0,700.0,13.0,0.0,0.0,15096.0,38.90625,18.0,1,0,223,1723.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,5.0,0.0,0,0,1,0,0.120496,3.0,8.570312,20.109375,57.50000,5.875000,4.554688,0.000898,683.5
2,5,298.2500,18,31698.0,8.0,74000.0,2,0,337.0,22.765625,0.0,675.0,11.0,0.0,0.0,4606.0,51.81250,27.0,0,0,175,0.0,0.0,0.0,0.0,21.0,5.0,3.0,0.0,0.0,0.0,4.0,0,1,16.0,0.0,0,0,1,0,0.048352,6.0,8.656250,20.234375,57.43750,5.882812,4.527344,0.000753,683.5
3,3,341.0000,4,46854.0,10.0,118000.0,1,4,148.0,17.203125,0.0,685.0,9.0,0.0,0.0,9948.0,52.59375,28.0,1,0,259,4.0,6.0,4.0,16.0,4.0,21.0,6.0,0.0,0.0,0.0,1.0,0,1,19.0,0.0,0,0,0,1,0.034674,3.0,8.515625,15.984375,41.71875,5.125000,5.046875,0.000552,720.0
4,3,101.0625,12,54.0,0.0,29000.0,2,10,301.0,32.156250,0.0,690.0,12.0,0.0,0.0,2942.0,32.00000,27.0,0,0,520,11.0,1.0,2.0,4.0,9.0,15.0,7.0,0.0,0.0,0.0,4.0,1,1,15.0,0.0,0,0,0,1,0.041822,6.0,8.414062,18.671875,55.06250,5.679688,4.601562,0.000731,688.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,3,860.5000,14,2659.0,7.0,72000.0,0,0,242.0,19.031250,0.0,710.0,14.0,0.0,0.0,9933.0,46.40625,15.0,1,0,112,0.0,0.0,6.0,6.0,2.0,13.0,10.0,0.0,0.0,0.0,3.0,0,1,1.0,0.0,0,0,0,1,0.143402,5.0,8.585938,19.234375,56.06250,5.773438,4.636719,0.000721,687.0
799996,3,532.0000,4,29205.0,10.0,99000.0,2,4,563.0,15.718750,1.0,720.0,7.0,0.0,0.0,20472.0,98.37500,42.0,0,0,379,33376.0,0.0,2.0,15.0,16.0,19.0,2.0,0.0,0.0,0.0,0.0,0,1,35.0,0.0,0,0,1,0,0.064478,9.0,8.515625,15.984375,41.71875,5.125000,5.046875,0.000552,720.0
799997,3,203.1250,13,2582.0,10.0,65000.0,2,0,47.0,12.109375,1.0,670.0,5.0,0.0,0.0,6381.0,51.90625,36.0,1,0,221,0.0,2.0,1.0,4.0,26.0,10.0,4.0,0.0,0.0,1.0,4.0,0,1,31.0,0.0,0,0,0,1,0.037499,1.0,8.523438,18.937500,55.15625,5.730469,4.625000,0.001122,688.0
799998,3,592.0000,4,151.0,10.0,96000.0,2,4,34.0,29.250000,0.0,675.0,16.0,0.0,0.0,69702.0,61.31250,37.0,1,0,323,4.0,0.0,7.0,10.0,6.0,22.0,8.0,0.0,0.0,0.0,5.0,0,1,21.0,0.0,0,0,1,0,0.074017,0.0,8.515625,15.984375,41.71875,5.125000,5.046875,0.000552,720.0


In [29]:
X_train['verificationStatus'] = X_train['verificationStatus'].astype('category')
X_train['applicationType'] = X_train['applicationType'].astype('category')
X_train['purpose'] = X_train['purpose'].astype('category')
X_train['initialListStatus'] = X_train['initialListStatus'].astype('category')

In [30]:
# 微调版
import lightgbm as lgb
"""使用lightgbm 5折交叉验证进行建模预测"""
cv_scores = []
for i, (train_index, valid_index) in enumerate(skf.split(X_train, y_train)):
    print('************************************ {} ************************************'.format(str(i+1)))
    X_train_split, y_train_split, X_val, y_val = X_train.iloc[train_index], y_train[train_index], X_train.iloc[valid_index], y_train[valid_index]
    
    train_matrix = lgb.Dataset(X_train_split, label=y_train_split)
    valid_matrix = lgb.Dataset(X_val, label=y_val)

    params = {
                    'boosting_type': 'gbdt',
                    'objective': 'binary',
                    'metric': 'auc',
                    'learning_rate': 0.01,
                    'num_leaves': 14,
                    'max_depth': 19,
                    'min_data_in_leaf': 37,
                    'min_child_weight':1.6,
                    'bagging_fraction': 0.98,
                    'feature_fraction': 0.69,
                    'bagging_freq': 96,
                    'reg_lambda': 9,
                    'reg_alpha': 7,
                    'min_split_gain': 0.4,
                    'nthread': 8,
                    'seed': 2020,
                    'silent': False,
        'categorical_feature':['name:verificationStatus', 'name:applicationType', 'name:initialListStatus', 'name:purpose'],
    }
    
    model = lgb.train(params, train_set=train_matrix, num_boost_round=14269, valid_sets=valid_matrix, verbose_eval=1000, early_stopping_rounds=200)
    val_pred = model.predict(X_val, num_iteration=model.best_iteration)
    
    cv_scores.append(roc_auc_score(y_val, val_pred))
    print(cv_scores)

print("lgb_scotrainre_list:{}".format(cv_scores))
print("lgb_score_mean:{}".format(np.mean(cv_scores)))
print("lgb_score_std:{}".format(np.std(cv_scores)))

************************************ 1 ************************************
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's auc: 0.723064
[2000]	valid_0's auc: 0.72577
[3000]	valid_0's auc: 0.726805
Early stopping, best iteration is:
[3775]	valid_0's auc: 0.727222
[0.7272223585061824]
************************************ 2 ************************************
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's auc: 0.722033
[2000]	valid_0's auc: 0.724631
[3000]	valid_0's auc: 0.725848
[4000]	valid_0's auc: 0.726581
[5000]	valid_0's auc: 0.727065
[6000]	valid_0's auc: 0.727403
[7000]	valid_0's auc: 0.727736
[8000]	valid_0's auc: 0.727969
[9000]	valid_0's auc: 0.728087
Early stopping, best iteration is:
[9581]	valid_0's auc: 0.728197
[0.7272223585061824, 0.7281974323062224]
************************************ 3 ************************************
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's auc:

In [31]:
# 存模型
import pickle
pickle.dump(model, open('./results/lgb_7283.pkl', 'wb'))

In [ ]:
""""""
base_params_lgb = {
                    'boosting_type': 'gbdt',
                    'objective': 'binary',
                    'metric': 'auc',
                    'learning_rate': 0.01,
                    'num_leaves': 14,
                    'max_depth': 19,
                    'min_data_in_leaf': 37,
                    'min_child_weight':1.6,
                    'bagging_fraction': 0.98,
                    'feature_fraction': 0.69,
                    'bagging_freq': 96,
                    'reg_lambda': 9,
                    'reg_alpha': 7,
                    'min_split_gain': 0.4,
                    'nthread': 8,
                    'seed': 2020,
                    'silent': False,
}

"""使用训练集数据进行模型训练"""
final_model_lgb = lgb.train(base_params_lgb, train_set=train_matrix, valid_sets=valid_matrix, num_boost_round=13000, verbose_eval=1000, early_stopping_rounds=200)

"""预测并计算roc的相关指标"""
val_pre_lgb = final_model_lgb.predict(X_val)
fpr, tpr, threshold = metrics.roc_curve(y_val, val_pre_lgb)
roc_auc = metrics.auc(fpr, tpr)
print('调参后lightgbm单模型在验证集上的AUC：{}'.format(roc_auc))
"""画出roc曲线图"""
plt.figure(figsize=(8, 8))
plt.title('Validation ROC')
plt.plot(fpr, tpr, 'b', label = 'Val AUC = %0.4f' % roc_auc)
plt.ylim(0,1)
plt.xlim(0,1)
plt.legend(loc='best')
plt.title('ROC')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
# 画出对角线
plt.plot([0,1],[0,1],'r--')
plt.show()

In [34]:
X_test['verificationStatus'] = X_test['verificationStatus'].astype('category')
X_test['applicationType'] = X_test['applicationType'].astype('category')
X_test['purpose'] = X_test['purpose'].astype('category')
X_test['initialListStatus'] = X_test['initialListStatus'].astype('category')

In [35]:
X_test['isDefault'] = model.predict(X_test.drop(columns=['id']))
X_test.head()

,id,term,installment,subGrade,employmentTitle,employmentLength,annualIncome,verificationStatus,purpose,postCode,dti,delinquency_2years,ficoRangeLow,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,n0,n4,n5,n6,n8,n9,n11,n12,n13,n14,employmentTitle_bin,fico,Acc,remainRec,remainRec_bin,pubRec_bin,homeOwnership_0,homeOwnership_1,debtRatio,postCode_bin,n6_mean,dti_mean,revolUtil_mean,n9_mean,n4_mean,n11_mean,ficoRangeLow_mean,isDefault
0,800000,3,458.25,8,7027.0,10.0,80000.0,0,0,163.0,10.562500,1.0,715.0,17.0,0.0,0.0,9846.0,30.703125,29.0,0,0,553,0.0,1.0,6.0,8.0,4.0,19.0,6.0,0.0,0.0,1.0,3.0,0,1,12.0,0.0,0,0,1,0,0.068742,4.0,8.359375,17.406250,51.03125,5.511719,4.746094,0.000935,697.5,0.648894
1,800001,5,472.25,15,60426.0,10.0,50000.0,0,2,235.0,21.406250,2.0,670.0,5.0,0.0,0.0,8946.0,56.593750,14.0,0,0,233,5.0,2.0,1.0,1.0,3.0,9.0,3.0,0.0,0.0,2.0,2.0,0,1,9.0,0.0,0,0,1,0,0.113314,5.0,8.593750,19.625000,56.06250,5.808594,4.625000,0.000818,686.0,0.735354
2,800002,3,446.00,19,23547.0,2.0,60000.0,2,0,526.0,33.500000,0.0,710.0,12.0,0.0,0.0,970.0,17.593750,43.0,1,0,172,0.0,0.0,1.0,1.0,36.0,6.0,4.0,0.0,0.0,0.0,7.0,0,1,31.0,0.0,0,0,0,1,0.089182,8.0,8.781250,20.156250,58.71875,5.933594,4.558594,0.001131,682.5,0.603149
3,800003,5,410.00,14,636.0,4.0,37000.0,1,4,248.0,13.953125,0.0,685.0,10.0,1.0,1.0,10249.0,52.312500,18.0,0,0,221,4.0,0.0,4.0,7.0,2.0,14.0,2.0,0.0,0.0,0.0,3.0,0,1,8.0,0.0,0,1,1,0,0.132979,5.0,8.585938,19.234375,56.06250,5.773438,4.636719,0.000721,687.0,0.652003
4,800004,3,1249.00,16,368446.0,0.0,80000.0,1,0,115.0,24.968750,0.0,685.0,19.0,0.0,0.0,33199.0,35.593750,22.0,0,0,240,0.0,0.0,9.0,11.0,3.0,18.0,11.0,0.0,0.0,0.0,1.0,0,1,3.0,0.0,0,0,0,1,0.187413,2.0,8.546875,19.921875,57.53125,5.882812,4.597656,0.001176,684.0,0.618568


In [36]:
res = X_test[['id', 'isDefault']]
res.head()

,id,isDefault
0,800000,0.648894
1,800001,0.735354
2,800002,0.603149
3,800003,0.652003
4,800004,0.618568


In [37]:
res.to_csv('./results/submit_1205_1.csv', index=False)

In [38]:
res.head()

,id,isDefault
0,800000,0.648894
1,800001,0.735354
2,800002,0.603149
3,800003,0.652003
4,800004,0.618568


In [41]:
a = pd.read_csv('./results/submit_1205_1.csv')
a[a['isDefault']>0.51].count()

id           184116
isDefault    184116
dtype: int64

In [42]:
data.head()

,id,term,installment,subGrade,employmentTitle,employmentLength,annualIncome,verificationStatus,isDefault,purpose,postCode,dti,delinquency_2years,ficoRangeLow,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,n0,n4,n5,n6,n8,n9,n11,n12,n13,n14,istest,employmentTitle_bin,fico,Acc,remainRec,remainRec_bin,pubRec_bin,homeOwnership_0,homeOwnership_1,debtRatio,postCode_bin,n6_mean,dti_mean,revolUtil_mean,n9_mean,n4_mean,n11_mean,ficoRangeLow_mean
0,0,5,918.0000,22,320.0,2.0,110000.0,2,1.0,1,137.0,17.046875,0.0,730.0,7.0,0.0,0.0,24178.0,48.90625,27.0,0,0,232,1.0,0.0,4.0,9.0,8.0,12.0,2.0,0.0,0.0,0.0,2.0,0,0,1,20.0,0.0,0,0,0,0,0.100142,3.0,8.851562,20.609375,59.15625,6.007812,4.605469,0.001111,682.0
1,1,5,462.0000,17,219843.0,5.0,46000.0,2,0.0,0,156.0,27.828125,0.0,700.0,13.0,0.0,0.0,15096.0,38.90625,18.0,1,0,223,1723.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,5.0,0.0,0,0,1,0,0.120496,3.0,8.570312,20.109375,57.50000,5.875000,4.554688,0.000898,683.5
2,2,5,298.2500,18,31698.0,8.0,74000.0,2,0.0,0,337.0,22.765625,0.0,675.0,11.0,0.0,0.0,4606.0,51.81250,27.0,0,0,175,0.0,0.0,0.0,0.0,21.0,5.0,3.0,0.0,0.0,0.0,4.0,0,0,1,16.0,0.0,0,0,1,0,0.048352,6.0,8.656250,20.234375,57.43750,5.882812,4.527344,0.000753,683.5
3,3,3,341.0000,4,46854.0,10.0,118000.0,1,0.0,4,148.0,17.203125,0.0,685.0,9.0,0.0,0.0,9948.0,52.59375,28.0,1,0,259,4.0,6.0,4.0,16.0,4.0,21.0,6.0,0.0,0.0,0.0,1.0,0,0,1,19.0,0.0,0,0,0,1,0.034674,3.0,8.515625,15.984375,41.71875,5.125000,5.046875,0.000552,720.0
4,4,3,101.0625,12,54.0,0.0,29000.0,2,0.0,10,301.0,32.156250,0.0,690.0,12.0,0.0,0.0,2942.0,32.00000,27.0,0,0,520,11.0,1.0,2.0,4.0,9.0,15.0,7.0,0.0,0.0,0.0,4.0,0,1,1,15.0,0.0,0,0,0,1,0.041822,6.0,8.414062,18.671875,55.06250,5.679688,4.601562,0.000731,688.5


In [47]:
important = pd.DataFrame({
        'column': X_train.columns,
        'importance': model.feature_importance(),
    }).sort_values(by='importance')[-25:-1]

In [50]:
most_important_fea = important['column']
data_important = data[most_important_fea]
data_important.head()

,n4_mean,employmentLength,openAcc,n4,n5,totalAcc,n14,title,subGrade,Acc,n8,n9,ficoRangeLow,n6,id,purpose,revolUtil,debtRatio,dti,postCode,installment,annualIncome,employmentTitle,revolBal
0,4.605469,2.0,7.0,4.0,9.0,27.0,2.0,1.0,22,20.0,12.0,2.0,730.0,8.0,0,1,48.90625,0.100142,17.046875,137.0,918.0000,110000.0,320.0,24178.0
1,4.554688,5.0,13.0,10.0,NaN,18.0,NaN,1723.0,17,5.0,NaN,NaN,700.0,NaN,1,0,38.90625,0.120496,27.828125,156.0,462.0000,46000.0,219843.0,15096.0
2,4.527344,8.0,11.0,0.0,0.0,27.0,4.0,0.0,18,16.0,5.0,3.0,675.0,21.0,2,0,51.81250,0.048352,22.765625,337.0,298.2500,74000.0,31698.0,4606.0
3,5.046875,10.0,9.0,4.0,16.0,28.0,1.0,4.0,4,19.0,21.0,6.0,685.0,4.0,3,4,52.59375,0.034674,17.203125,148.0,341.0000,118000.0,46854.0,9948.0
4,4.601562,0.0,12.0,2.0,4.0,27.0,4.0,11.0,12,15.0,15.0,7.0,690.0,9.0,4,10,32.00000,0.041822,32.156250,301.0,101.0625,29000.0,54.0,2942.0


In [51]:
data_important.to_csv('./results/data_important.csv', index=False)

In [54]:
data.drop(columns=['debtRatio'])

,id,term,installment,subGrade,employmentTitle,employmentLength,annualIncome,verificationStatus,isDefault,purpose,postCode,dti,delinquency_2years,ficoRangeLow,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,n0,n4,n5,n6,n8,n9,n11,n12,n13,n14,istest,employmentTitle_bin,fico,Acc,remainRec,remainRec_bin,pubRec_bin,homeOwnership_0,homeOwnership_1,postCode_bin,n6_mean,dti_mean,revolUtil_mean,n9_mean,n4_mean,n11_mean,ficoRangeLow_mean
0,0,5,918.0000,22,320.0,2.0,110000.0,2,1.0,1,137.0,17.046875,0.0,730.0,7.0,0.0,0.0,24178.0,48.90625,27.0,0,0,232,1.0,0.0,4.0,9.0,8.0,12.0,2.0,0.0,0.0,0.0,2.0,0,0,1,20.0,0.0,0,0,0,0,3.0,8.851562,20.609375,59.15625,6.007812,4.605469,0.001111,682.0
1,1,5,462.0000,17,219843.0,5.0,46000.0,2,0.0,0,156.0,27.828125,0.0,700.0,13.0,0.0,0.0,15096.0,38.90625,18.0,1,0,223,1723.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,5.0,0.0,0,0,1,0,3.0,8.570312,20.109375,57.50000,5.875000,4.554688,0.000898,683.5
2,2,5,298.2500,18,31698.0,8.0,74000.0,2,0.0,0,337.0,22.765625,0.0,675.0,11.0,0.0,0.0,4606.0,51.81250,27.0,0,0,175,0.0,0.0,0.0,0.0,21.0,5.0,3.0,0.0,0.0,0.0,4.0,0,0,1,16.0,0.0,0,0,1,0,6.0,8.656250,20.234375,57.43750,5.882812,4.527344,0.000753,683.5
3,3,3,341.0000,4,46854.0,10.0,118000.0,1,0.0,4,148.0,17.203125,0.0,685.0,9.0,0.0,0.0,9948.0,52.59375,28.0,1,0,259,4.0,6.0,4.0,16.0,4.0,21.0,6.0,0.0,0.0,0.0,1.0,0,0,1,19.0,0.0,0,0,0,1,3.0,8.515625,15.984375,41.71875,5.125000,5.046875,0.000552,720.0
4,4,3,101.0625,12,54.0,0.0,29000.0,2,0.0,10,301.0,32.156250,0.0,690.0,12.0,0.0,0.0,2942.0,32.00000,27.0,0,0,520,11.0,1.0,2.0,4.0,9.0,15.0,7.0,0.0,0.0,0.0,4.0,0,1,1,15.0,0.0,0,0,0,1,6.0,8.414062,18.671875,55.06250,5.679688,4.601562,0.000731,688.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,999995,3,229.6250,7,330967.0,7.0,30000.0,2,NaN,4,36.0,32.906250,0.0,685.0,8.0,0.0,0.0,5021.0,54.00000,17.0,1,0,181,52672.0,0.0,2.0,6.0,11.0,6.0,2.0,0.0,0.0,0.0,4.0,1,0,1,9.0,0.0,0,0,0,1,0.0,8.328125,17.093750,49.40625,5.390625,4.761719,0.000634,700.0
199996,999996,3,183.2500,2,38930.0,1.0,56000.0,0,NaN,0,261.0,20.375000,0.0,720.0,25.0,0.0,0.0,10163.0,35.90625,27.0,0,0,170,0.0,0.0,5.0,5.0,14.0,13.0,6.0,0.0,0.0,0.0,0.0,1,0,1,2.0,0.0,0,0,0,1,5.0,8.765625,15.203125,34.81250,4.945312,5.300781,0.000040,736.5
199997,999997,5,339.5000,14,282016.0,8.0,80000.0,2,NaN,4,46.0,15.117188,0.0,675.0,21.0,0.0,0.0,31401.0,86.00000,63.0,0,0,228,4.0,0.0,9.0,21.0,18.0,42.0,13.0,0.0,0.0,0.0,0.0,1,0,1,42.0,0.0,0,0,0,0,1.0,8.585938,19.234375,56.06250,5.773438,4.636719,0.000721,687.0
199998,999998,3,289.5000,17,97.0,4.0,190000.0,0,NaN,0,99.0,9.328125,0.0,690.0,11.0,0.0,0.0,28493.0,64.00000,24.0,0,0,184,0.0,0.0,4.0,12.0,5.0,19.0,6.0,0.0,0.0,0.0,2.0,1,0,1,13.0,0.0,0,0,0,1,2.0,8.570312,20.109375,57.50000,5.875000,4.554688,0.000898,683.5


In [17]:
data.to_csv('./results/data.csv', index=False)